## Coffee Roasting neural network with NumPy

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Normalization # type: ignore
from lab_utils_common import sigmoid
from lab_coffee_utils import load_coffee_data, plt_roast

# Allows us to manage and control the log messages
import logging
# Sets the logging level for tensorflow to show only errors
logging.getLogger("tensorflow").setLevel(logging.ERROR)
# Controls the verbosity of tensorflow autograph i.e; python code to tensorflow graph code
tf.autograph.set_verbosity(0)
# Disables internal logging or output messages

In [ ]:
x_train, y_train=load_coffee_data()
print(x_train.shape, y_train.shape)

In [ ]:
plt_roast(x_train, y_train)

In [ ]:
print(f"Temperature max, min pre normalization: {np.max(x_train[:,0]):0.2f}, {np.min(x_train[:,0]):0.2f}")
print(f"Duration max, min pre normalization: {np.max(x_train[:,1]):0.2f}, {np.min(x_train[:,1]):0.2f}")

norm_l=Normalization(axis=-1)
norm_l.adapt(x_train)
x_norm=norm_l(x_train)

print(f"Temperature max, min post normalization: {np.max(x_norm[:,0]):0.2f}, {np.min(x_norm[:,0]):0.2f}")
print(f"Duration max, min post normalization: {np.max(x_norm[:,1]):0.2f}, {np.min(x_norm[:,1]):0.2f}")

### NumPy Model

In [ ]:
def dense_func(a_in, W, b, g):
  units=W.shape[1]
  a_out=np.zeros(units)

  for i in range(units):
    w=W[:, i]
    z=np.dot(a_in, w)+b[i]
    a_out[i]=g(z)

  return a_out

In [ ]:
def sequential_func(x, W1, b1, W2, b2):
  a1=dense_func(x, W1, b1, sigmoid)
  a2=dense_func(a1, W2, b2, sigmoid)

  return a2

In [ ]:
W1_tmp=np.array([
  [-8.93,  0.29, 12.9 ], 
  [-0.1,  -7.32, 10.81]])
b1_tmp=np.array([-9.82, -9.28,  0.96])

W2_tmp=np.array([
  [-31.18], 
  [-27.59], 
  [-32.56]])
b2_tmp=np.array([15.41])

In [ ]:
def predict_func(x, W1, b1, W2, b2):
  m=x.shape[0]
  p=np.zeros((m, 1))

  for i in range(m):
    p[i, 0]=sequential_func(x[i], W1, b1, W2, b2).item()

  return p

In [ ]:
x_test=np.array([
  [200, 13.9],
  [200, 17]
])

x_test_norm=norm_l(x_test)
predictions=predict_func(x_test_norm, W1_tmp, b1_tmp, W2_tmp, b2_tmp)
print(f"Predictions: {predictions}")

In [ ]:
y_pred=np.zeros_like(predictions)

for i in range(len(predictions)):
  if predictions[i]>=0.5:
    y_pred[i]=1
  else:
    y_pred[i]=0

print(f"Decisions: {y_pred}")

In [ ]:
y_hat=(predictions>=0.5).astype(int)
print(f"Decisions: {y_hat}")